https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

The goal is to generate names using the RNN architecture designed in the 1-RNN.

# Imports

In [162]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import re
import random
import os

import typing

# Loading data

In [138]:
categories = []
category_lines = {}

for (dirpath, dirnames, filenames) in os.walk('data/names'):
    for (i, filename) in enumerate([dirpath + os.sep + f for f in filenames]):
        category = os.path.splitext(os.path.basename(filename))[0]
        categories.append(category)
        lines = open(filename, encoding='utf-8').read().lower().strip().split('\n')
        category_lines[category] = lines
        
n_categories = len(category_lines)

In [139]:
def input_tensor(name: str) -> torch.Tensor:
    tensor = torch.zeros(len(name), 1, 512)
    for (i, c) in enumerate(name):
        tensor[i] = F.one_hot(torch.Tensor([ord(c)]).long(), num_classes=512)
        
    return tensor

In [140]:
def category_to_tensor(category: str) -> torch.Tensor:
    return F.one_hot(torch.Tensor([categories.index(category)]).long(), num_classes=n_categories)

In [141]:
def target_tensor(name: str) -> torch.Tensor:
    tensor = torch.zeros(len(name), 1, 512)
    for (i, c) in enumerate(name):
        if i > 0:
            tensor[i - 1] = F.one_hot(torch.Tensor([ord(c)]).long(), num_classes=512)
            
    tensor[len(name) - 1] = F.one_hot(torch.Tensor([512 - 1]).long(), num_classes=512)
    
    return tensor

# Neural network module

In [142]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_categories):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), dim=1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), dim=1)
        output = self.softmax(self.dropout(self.o2o(output_combined)))
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [143]:
n_hidden = 128
rnn = RNN(512, n_hidden, 512, len(categories))

# Training

In [170]:
def get_training_example() -> typing.Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    category = random.choice(categories)
    line = random.choice(category_lines[category])
    category_t = category_to_tensor(category)
    line_t = input_tensor(line)
    target_t = target_tensor(line)
    return category_t, line_t, target_t

In [145]:
criterion = nn.NLLLoss()

n_iters = 100000
learning_rate = 0.0005

def train(category_tensor, input_tensor, target_tensor):
    target_tensor.unsqueeze_(1)
    hidden = rnn.init_hidden()
    
    rnn.zero_grad()
    
    loss = torch.Tensor([0])
    
    for i in range(input_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_tensor[i], hidden)
        l = criterion(output, target_tensor[i].squeeze(0))
        loss += l
        
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return output, loss.item() / input_tensor.size(0)

In [179]:
current_loss = 0
for iter in range(1, n_iters + 1):
    category_tensor, line_tensor, target_t = get_training_example()
    output, loss = train(category_tensor, line_tensor, target_t)
    current_loss += loss

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

# Evaluation